In [ ]:
# Run this script to compare and  execute all database objects on the synapse serverless sql server
import sys
import pandas as pd
from datetime import datetime as dt
sys.path.insert(0, r"examplecode\python\functions")
from functions.sqlauth import *
from functions.synserverlessobjects import *
from functions.dataframeoperations import *

server = 'target-synapse-server-ondemand.sql.azuresynapse.net'

#get local settings
username, repo_path = get_personal_settings()
base_wd = 'C:\\Users\\Arthur.Steijn\\source\\repos\\synapse-serverless-sql-cicd-python\\synapse-serverless-objects\\'


environment_reference_d = 'development-server'
environment_reference_p = 'production-server'

# set to variable to true, to execute statements on server
execute_var = 'False'

def main():
    """ Function to connect to an azure synapse ondemand server 
        and execute all views, stored procedures and functions on their databases """
    
    #authenticate to the server with azure ad account
    print('Creata a connection and cursor to the synapse server')
    cursor, cnxn =  dbcnxn(server, username)
   
    #create df from repo
    print('Create dataframe from repo scripts. Folder is set to: ' + repo_path)
    df_repo = create_df_from_repo(repo_path, base_wd, localind='True')
    print(df_repo)
    
    #get all databases on server
    print('Get all databases from the server, current databases are: ')   
    databases = get_databases(cursor)
    for row in databases:
        print(row[0])

    #get all queries
    print('Get queries to export synapseserver objects to a dataframe: ')   
    sql_views, sql_sps, sql_fn = get_queries()

    #get all objects from server
    df_target = pd.DataFrame()
    for row in databases:
        database = row[0]
        if 'demo_db' not in database:
            df_views = create_target_df(sql_views, cnxn, cursor, database, 'VIEW')
            df_target = df_target.append(df_views)
            df_sps = create_target_df(sql_sps, cnxn, cursor, database, 'PROCEDURE')
            df_target = df_target.append(df_sps)
            df_fns = create_target_df(sql_fn, cnxn, cursor, database, 'FUNCTION')
            df_target = df_target.append(df_fns)

    #clean target dataframe
    df_target['definition'] = df_target['definition'].transform(clean_sql_func_df)
    df_target['Env'] = "target" 

    print(df_repo.columns)
    print(df_target.columns)

    #merge data frames
    print('Prepare and Clean dataframe')
    df_merged = merge_dataframes(df_repo, df_target)
    df_prepared = clean_and_prepare(df_merged)

    print(df_prepared)
 
    #For Checking Purposes we create a csv that we can publish in the pipeline
    filename = 'prepared_df'+dt.now().strftime("%Y%m%d_%H%M%S")+'.txt'
    with open(filename, 'w', newline='') as f:
        df_prepared.to_csv(f, sep='|')

    filename = 'merged_df'+dt.now().strftime("%Y%m%d_%H%M%S")+'.txt'
    with open(filename, 'w', newline='') as f:
        df_merged.to_csv(f, sep='|')

    print('execute_var is set to: ' + execute_var)
    if(execute_var == 'True'):
        
        for index, row in df_prepared.iterrows():
            database = row['DBName']
            statement =  row['Execute_Statement']
            schemaname = row['SchemaName']
            objectname = row['ObjectName']
            print('index:' + str(index) +' Executing for: '+ database +'.'+ schemaname +'.'+ objectname)
            execute_statement(cursor, database, statement)
    else:
        print('no database executions')

    # close connection
    cnxn.commit()
    cursor.close()

if __name__ == "__main__":
    main()